Isssue the magic command %status to test the status of the neptune cluster created. The notebook is already attached to the neptune cluster

In [2]:
%status

{'status': 'healthy',
 'startTime': 'Fri Oct 16 08:35:42 UTC 2020',
 'dbEngineVersion': '1.0.2.2.R5',
 'role': 'writer',
 'gremlin': {'version': 'tinkerpop-3.4.3'},
 'sparql': {'version': 'sparql-1.1'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'}}

The Neptune workbench has a %seed command that you can use to load the sample data that the queries in this post will use. Simply create a new cell in your notebook, enter %seed and run the cell. You will be promted for the "Language" (graph type) and "Data set" you want to use.

In [3]:
%seed

Dropdown(description='Language:', options=('', 'Gremlin', 'SPARQL'), value='')

Dropdown(description='Data set:', layout=Layout(visibility='hidden'), options=(), value=None)

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

A graph query is often referred to as a traversal as that is what we are in fact doing. We are traversing the graph from a starting point to an ending point. Traversals consist of one or more steps (essentially methods) that are chained together. Every step is either a map-step (transforming the objects in the stream), a filter-step (removing objects from the stream), or a sideEffect-step (computing statistics about the stream).

The first step is to create a graph traversal source object for our loaded graph.

g = graph.traversal()

Count the number of vertices and edges in the graph. The graph contains the loaded airlines data set.

In [7]:
%%gremlin
g.V().count()

In [8]:
%%gremlin
g.E().count()

#### Return the vertex that represents the Bengaluru (BLR) airport

In [9]:
%%gremlin
g.V().hasLabel('airport').has('code','BLR')

#### Get the properties of BLR airport

In [10]:
%%gremlin
g.V().has('airport','code','BLR').valueMap()

#### How many airports are in the graph

In [11]:
%%gremlin
g.V().hasLabel('airport').count()

#### Count Vertices by groups


In [12]:
%%gremlin
g.V().groupCount().by(label)

#### How many airports are in each country

In [16]:
%%gremlin
g.V().hasLabel('airport').groupCount().by('country')

In [19]:
%%gremlin
g.V().hasLabel('airport').groupCount().by('country').select('US','UK','DK','IN')

#### Starting to walk the graph
The term walking the graph is used to describe moving from one vertex to another vertex via an edge. Typically when using the phrase walking a graph the intent is to describe starting at a vertex traversing one or more vertices and edges and ending up at a different vertex or sometimes, back where you started in the case of a circular walk. The journey we take while on our walk is often referred to as our path. There are also cases when all you want to do is return edges or some combination of vertices and edges as the result of a query and Gremlin allows this as well.

The table below gives a brief summary of all the steps that can be used to walk or traverse a graph using Gremlin.

|Step   | Where to move next
 :-----  ---------------
out* |	Outgoing adjacent vertices.
in*	 | Incoming adjacent vertices.
both*|	Both incoming and outgoing adjacent vertices.
outE*|	Outgoing incident edges.
inE* |	Incoming incident edges.
bothE*|	Both outgoing and incoming incident edges.
outV |	Outgoing vertex.
inV |	Incoming vertex.
otherV*|	The vertex that was not the vertex we came from.
The steps labelled with an * can optionally take the name of one or more edge labels as a parameter. If omitted, all relevant edges will be traversed

#### What are the cities i can fly from Bengaluru

In [35]:
%%gremlin
g.V().has('airport','code','BLR').out().values('city')

In [32]:
%%gremlin
g.V().has('airport','code','BLR').out().out().dedup().not(cyclicPath()).path().by('city')

#### Flights from India to US

In [51]:
%%gremlin
g.V().has('airport','country','IN').out().has('country','US').path().by('city')

In [40]:
%%gremlin
g.V().hasLabel('country').count()

In [41]:
%%gremlin
g.V().hasLabel('country').valueMap()

In [47]:
%%gremlin
g.V().has('airport','country','IN').count()